In [8]:
#import
import os
import numpy as np
import pandas as pd
import re
import jieba
import jieba.posseg as pseg

In [2]:
df_texts = pd.read_csv('df_texts_with_index.csv') #reading the dataframe with texts

In [3]:
with open('UsrWords.txt', 'r', encoding='utf8') as f:
    custom_noun = f.readlines()
    for noun in custom_noun:
        jieba.add_word(noun.replace('\n', ''), freq=None, tag='n')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.479 seconds.
Prefix dict has been built successfully.


In [4]:
df_texts['words'] = df_texts.texts.apply(jieba.lcut)
df_texts.head()

,id,texts,words
0,1,个临时发往武汉的口罩\n个临时发往武汉的口罩个临时发往武汉的口罩个临时发往武汉的口罩吴呈杰报...,"[个, 临时, 发, 往, 武汉, 的, 口罩, \n, 个, 临时, 发, 往, 武汉, ..."
1,2,武汉隔离疫区信息孤岛与一辆鄂车的漂流\n武汉隔离疫区信息孤岛与一辆鄂车的漂流武汉隔离疫区信息...,"[武汉, 隔离, 疫区, 信息, 孤岛, 与, 一辆, 鄂车, 的, 漂流, \n, 武汉,..."
2,3,我家离华南海鲜市场很近返乡封城过年一位武汉大学生的过去一周\n我家离华南海鲜市场很近返乡封城...,"[我家, 离, 华南, 海鲜, 市场, 很近, 返乡, 封城, 过年, 一位, 武汉, 大学..."
3,4,疫情危机中不被看见的人们武汉周边城市百姓的自救行动\n疫情危机中不被看见的人们武汉周边城市百...,"[疫情, 危机, 中, 不, 被, 看见, 的, 人们, 武汉, 周边城市, 百姓, 的, ..."
4,5,孝感前线医生武汉更难我们下面不好意思提要求\n孝感前线医生武汉更难我们下面不好意思提要求孝感...,"[孝感, 前线, 医生, 武汉, 更难, 我们, 下面, 不好意思, 提, 要求, \n, ..."


The cutting of the words before extracting the places names (pseg analysis) is not actually necessary but it makes it possible to make a quick comparison if necessary. Let's do this analysis now.

In [9]:
#function to keep words based on jieba tagging, here we only keep places (ns)

def keep_words(s, to_keep=['ns']):
    s_ = []
    pos_list = pseg.lcut(s)
    for item in pos_list:
        if item.flag in to_keep:
            s_.append(item.word)
    return s_

In [10]:
df_texts['places'] = df_texts.texts.apply(keep_words)

In [11]:
df_texts

,id,texts,words,places
0,1,个临时发往武汉的口罩\n个临时发往武汉的口罩个临时发往武汉的口罩个临时发往武汉的口罩吴呈杰报...,"[个, 临时, 发, 往, 武汉, 的, 口罩, \n, 个, 临时, 发, 往, 武汉, ...","[武汉, 武汉, 武汉, 武汉, 武汉, 广州, 中康, 中国, 美国, 河南, 中断, 江..."
1,2,武汉隔离疫区信息孤岛与一辆鄂车的漂流\n武汉隔离疫区信息孤岛与一辆鄂车的漂流武汉隔离疫区信息...,"[武汉, 隔离, 疫区, 信息, 孤岛, 与, 一辆, 鄂车, 的, 漂流, \n, 武汉,...","[武汉, 鄂车, 武汉, 鄂车, 武汉, 鄂车, 武汉, 鄂车, 武汉, 鄂州, 东西, 武..."
2,3,我家离华南海鲜市场很近返乡封城过年一位武汉大学生的过去一周\n我家离华南海鲜市场很近返乡封城...,"[我家, 离, 华南, 海鲜, 市场, 很近, 返乡, 封城, 过年, 一位, 武汉, 大学...","[华南, 海鲜, 武汉, 华南, 海鲜, 武汉, 华南, 海鲜, 武汉, 华南, 海鲜, 武..."
3,4,疫情危机中不被看见的人们武汉周边城市百姓的自救行动\n疫情危机中不被看见的人们武汉周边城市百...,"[疫情, 危机, 中, 不, 被, 看见, 的, 人们, 武汉, 周边城市, 百姓, 的, ...","[武汉, 周边城市, 武汉, 周边城市, 武汉, 周边城市, 武汉, 周边城市, 武汉, 华..."
4,5,孝感前线医生武汉更难我们下面不好意思提要求\n孝感前线医生武汉更难我们下面不好意思提要求孝感...,"[孝感, 前线, 医生, 武汉, 更难, 我们, 下面, 不好意思, 提, 要求, \n, ...","[武汉, 武汉, 武汉, 武汉, 武安, 孝感市, 武汉, 武汉, 武汉, 武汉, 武安, ..."
5,6,恐慌很正常当疫情结束才是心理问题高发期\n恐慌很正常当疫情结束才是心理问题高发期恐慌很正常当...,"[恐慌, 很, 正常, 当, 疫情, 结束, 才, 是, 心理, 问题, 高发期, \n, ...","[汶川, 深圳, 新区, 滑坡, 武汉, 武汉, 英国, 武汉, 武汉, 广州, 北京, 上..."
6,7,外卖骑手出租车司机快递员志愿者封城九日他们维护着武汉的血脉\n外卖骑手出租车司机快递员志愿者...,"[外卖, 骑手, 出租车, 司机, 快递, 员, 志愿者, 封城, 九日, 他们, 维护, ...","[武汉, 武汉, 武汉, 武汉, 武汉, 城市, 城市, 武汉, 城市, 武汉, 武汉, 沧..."
7,8,疫情中的心理援助寻找局部的安全\n疫情中的心理援助寻找局部的安全疫情中的心理援助寻找局部的安...,"[疫情, 中, 的, 心理, 援助, 寻找, 局部, 的, 安全, \n, 疫情, 中, 的...","[上西, 汶川, 天津港, 中国, 湖北, 东西, 东西, 天津港, 天津市, 天津港, 天..."
8,9,鸡在瘟疫蔓延时\n鸡在瘟疫蔓延时鸡在瘟疫蔓延时鸡在瘟疫蔓延时蔚琳颖迪呈杰敏洁报道报道记录人物...,"[鸡, 在, 瘟疫, 蔓延, 时, \n, 鸡, 在, 瘟疫, 蔓延, 时鸡, 在, 瘟疫,...","[河南, 漯河, 舞阳县, 河南, 河南, 佳宁, 河南, 河南, 荥阳, 河南, 安徽, ..."
9,10,武汉封城后的宠物营救计划人类犯错的后果不该由它们承担\n武汉封城后的宠物营救计划人类犯错的后...,"[武汉, 封城后, 的, 宠物, 营救, 计划, 人类, 犯错, 的, 后果, 不该, 由,...","[武汉, 武汉, 武汉, 武汉, 武汉, 城市, 武汉, 武汉, 上门, 武汉, 武汉, 武..."


Now we have extracted all the places, we need to count them. To do that we need first make one list out of the list of list.

In [14]:
from  itertools import chain

In [22]:
fp = pd.Series(list(chain.from_iterable(df_texts['places'])))
fp.head(5)

0    武汉
1    武汉
2    武汉
3    武汉
4    武汉
dtype: object

In [38]:
cp = fp.value_counts()
cp.head(30)

武汉     42303
中国     19574
湖北      9605
武汉市     7677
美国      7590
北京      6113
意大利     5875
城市      4837
上海      4503
湖北省     3886
韩国      3689
日本      3396
伊朗      2959
东西      2589
英国      2544
欧洲      2252
香港      1932
海鲜      1895
德国      1873
华南      1866
广州      1576
回国      1524
钟南山     1394
深度      1384
黄冈      1332
广东      1228
深圳      1168
法国      1122
神山       985
汉口       982
dtype: int64

Wuhan is definitely on top of that list and by a lot of counts. However, there are also a lot of trash: 东西 is not a place f.e. If we want to use this places to put them on a map we need to edit them manually first. We are going to make the list as a dataframe, save the dataframe and edit it with libre office calc.

In [39]:
dcp= cp.to_frame().reset_index()

In [40]:
dcp.rename(columns = {'index':'places', 0: 'freq'}, inplace = True)

In [41]:
dcp.head()

,places,freq
0,武汉,42303
1,中国,19574
2,湖北,9605
3,武汉市,7677
4,美国,7590


In [42]:
#dcp.to_csv (r'dataframes/3corpus_places_freq.csv', index = True, header=True) #with index

I kept the first 100 values, tried to remove values which weren't real places and remove duplicates ( ex: 上海, 上海市) but kept a certain area in a city different from the city, or the city different from the province.

In [190]:
dcpf= pd.read_csv('dataframes/2corpus_places_freq.csv') #reading the corrected dataframe

In [191]:
dcpf.columns

Index(['Unnamed: 0', 'places', 'freq'], dtype='object')

In [192]:
dcpf = dcpf.drop(columns=['Unnamed: 0'])

In [193]:
dcpf = dcpf.sort_values(by = 'freq', ascending = False ).reset_index(drop=True)
dcpf

,places,freq
0,武汉,49980
1,中国,19574
2,湖北,13491
3,美国,8025
4,北京,6723
5,意大利,5875
6,上海,5273
7,韩国,3689
8,日本,3396
9,伊朗,2959


In [194]:
from geopy.geocoders import Nominatim
import time

In [195]:
geolocator = Nominatim(user_agent="Your_Name")

def ex_lat(address):
    time.sleep(1) #adding 1 second delay before each request to avoid making too many request to nominatim servers
    location = geolocator.geocode(address)
    return location.latitude
    
def ex_long(address):
    time.sleep(1)
    location = geolocator.geocode(address)
    return location.longitude

In [196]:
dcpf['lat'] = dcpf['places'].apply(ex_lat)
dcpf['long'] = dcpf['places'].apply(ex_long)
dcpf

,places,freq,lat,long
0,武汉,49980,30.595105,114.299935
1,中国,19574,35.000074,104.999927
2,湖北,13491,31.151725,112.878322
3,美国,8025,39.783730,-100.445882
4,北京,6723,39.906217,116.391276
5,意大利,5875,42.638426,12.674297
6,上海,5273,31.232276,121.469207
7,韩国,3689,36.638392,127.696119
8,日本,3396,36.574844,139.239418
9,伊朗,2959,32.647531,54.564352


In [197]:
# import folium
import folium

In [198]:
# Make an empty map
m = folium.Map(location=[32.0609736, 118.7916458], tiles="OpenStreetMap", zoom_start=4)

# Show the map
m

In [199]:
for places, freq, lat, long in zip(
    dcpf.places, 
    dcpf.freq, 
    dcpf.lat,
    dcpf.long):
    
    label= '{}, {}'.format(places, freq) # Define label here to reuse after
    
    m.add_child(
        folium.Circle(
            [lat,long],
            label = label,
            popup = label,
            radius = float(freq)*7, # define how big is the circle
            color = 'crimson',
            fill = True,
            fill_color = 'crimson',
            fill_opacity = 0.3
        ).add_to(m)
    )

In [200]:
m

With this map, we could see that there was still a few problem with our data. 中南 South central Chine and 中东 Middle east both appear in North Korea.

In [121]:
print(geolocator.geocode('中东'))
print(geolocator.geocode('الشرق الاوسط'))
#only giving a hint with viewbox actually help a little to locate middle east properly,
#we will have to enter the coordinates manually

geolocator.geocode('Middle East', viewbox = ((28,30),(41,43)) ) 


중동, 부천시, 오정구, 대한민국
الشرق الاوسط, باب الخان(امام حسن), محافظة كربلاء, كربلاء, ناحية مرکز قضاء الکربلاء, قضاء کربلاء, محافظة كربلاء, 56001, العراق / عێراق


Location(Middle East, Qalqilya, قلقيلية, منطقة أ, الضفة الغربية, 110, Palestinian Territory, (32.1862321, 34.9771488, 0.0))

In [123]:
print(geolocator.geocode('中南'))
print(geolocator.geocode('中南地区'))

충청남도, 대한민국
충청남도, 대한민국


No success here as well. Let's just enter manually the coordinate of Changsha, which is more or less in the center of the area voir wiki: https://fr.wikipedia.org/wiki/Zhongnan coordinates: 29.00244602807003, 112.96734232025437

However, Changsha is also in the dataframe and if two coordinates are the same, the map only keep the last one. We need to pay attention to that and enter coordinates nearby but not the same.

加州 JiaZhou California shows in Japan, let's just rename it in english on the list.

In [201]:
pd.set_option('display.max_rows', None)
dcpf

,places,freq,lat,long
0,武汉,49980,30.595105,114.299935
1,中国,19574,35.000074,104.999927
2,湖北,13491,31.151725,112.878322
3,美国,8025,39.783730,-100.445882
4,北京,6723,39.906217,116.391276
5,意大利,5875,42.638426,12.674297
6,上海,5273,31.232276,121.469207
7,韩国,3689,36.638392,127.696119
8,日本,3396,36.574844,139.239418
9,伊朗,2959,32.647531,54.564352


Let's edit manually our dataframe.

In [202]:
dcpf.at[24, 'lat'] = '25.8991844' #edit 中东
dcpf.at[24, 'long'] = '25.7454072' #edit 中东

dcpf.at[29, 'lat'] = '27.865395854384015' #edit 中南
dcpf.at[29, 'long'] = '112.37254013136757' #edit 中南

dcpf.at[64, 'lat'] = '19.54918' #edit 北非洲
dcpf.at[64, 'long'] = '-7.7429868' #edit 北非洲

dcpf

,places,freq,lat,long
0,武汉,49980,30.595105,114.299935
1,中国,19574,35.000074,104.999927
2,湖北,13491,31.151725,112.878322
3,美国,8025,39.783730,-100.445882
4,北京,6723,39.906217,116.391276
5,意大利,5875,42.638426,12.674297
6,上海,5273,31.232276,121.469207
7,韩国,3689,36.638392,127.696119
8,日本,3396,36.574844,139.239418
9,伊朗,2959,32.647531,54.564352


We can create a new column for easier readibility while displaying the popup. The column freq is used for computing the radius of the circle and freq2 for display.

In [231]:
def coupe(number):
    x = "{:,}".format(number)
    return x

In [232]:
dcpf['freq2'] = dcpf.freq.apply(coupe)
dcpf

,places,freq,lat,long,freq2
0,武汉,49980,30.595105,114.299935,"49,980"
1,中国,19574,35.000074,104.999927,"19,574"
2,湖北,13491,31.151725,112.878322,"13,491"
3,美国,8025,39.783730,-100.445882,"8,025"
4,北京,6723,39.906217,116.391276,"6,723"
5,意大利,5875,42.638426,12.674297,"5,875"
6,上海,5273,31.232276,121.469207,"5,273"
7,韩国,3689,36.638392,127.696119,"3,689"
8,日本,3396,36.574844,139.239418,"3,396"
9,伊朗,2959,32.647531,54.564352,"2,959"


In [245]:
#reset the map
m = folium.Map(location=[32.0609736, 118.7916458], tiles="OpenStreetMap", zoom_start=4)

#add a title
title = "First 100 places' occurence in nCovMemory corpus"
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(title)   


m.get_root().html.add_child(folium.Element(title_html))

#write again the circles with the changes made
for places, freq, lat, long, freq2 in zip(
    dcpf.places, 
    dcpf.freq, 
    dcpf.lat,
    dcpf.long,
    dcpf.freq2):
    
    label= '{}: {}'.format(places, freq2) # Define label here to reuse after
    
    m.add_child(
        folium.Circle(
            [lat,long],
            label = label,
            popup = label,
            radius = float(freq)*7, # define how big is the circle
            color = 'crimson',
            fill = True,
            fill_color = 'crimson',
            fill_opacity = 0.3
        ).add_to(m)
    )

In [246]:
m

In [247]:
#save the map
m.save("maps/places_1st100freq.html")